In [ ]:
import numpy as np
import pandas as pd
import pickle
import json

In [ ]:
dataset = pd.read_csv('Training.csv')

In [ ]:
model=pickle.load(open('models.dot','rb'))

In [ ]:
dataset1=dataset.drop(['prognosis'],axis=1)

In [ ]:
def inputTaker(dataset1):
  temp=pd.DataFrame(dataset1.sum(axis=0),columns=['frequency'])
  temp=temp.sort_values('frequency',ascending=False)
  choices=temp.head(10).index
  print('Select Symptoms(Comma Separated):')
  for i,v in enumerate(choices):
    print('['+str(i)+']'+' '+str(' '.join(v.split('_'))),end='\t')
  print('[-1] None')
  inputLine=input()
  inputLine=[int(x) for x in inputLine.split(',')]
  inputArr=dict()
  if -1 in inputLine:
    for i,v in enumerate(choices):
      inputArr[v]=0
  else:
    for i,v in enumerate(choices):
      if i in inputLine:
        inputArr[v]=1
  qry = ' and '.join(["{} == {}".format(k,v) for k,v in inputArr.items()])
  tempDataset=dataset1.query(qry)
  tempDataset=tempDataset.drop(choices,axis=1)
  if(tempDataset.shape[0]>1):
    inputArr.update(inputTaker(tempDataset))
  return inputArr

In [ ]:
inputArr=inputTaker(dataset1)

Select Symptoms(Comma Separated):
[0] fatigue	[1] vomiting	[2] high fever	[3] loss of appetite	[4] nausea	[5] headache	[6] abdominal pain	[7] yellowish skin	[8] yellowing of eyes	[9] chills	[-1] None
0,2,5,1
Select Symptoms(Comma Separated):
[0] belly pain	[1] diarrhoea	[2] constipation	[3] back pain	[4] pain behind the eyes	[5] toxic look (typhos)	[6] malaise	[7] skin rash	[8] joint pain	[9] red spots over body	[-1] None
3,7
Select Symptoms(Comma Separated):
[0] muscle pain	[1] itching	[2] nodal skin eruptions	[3] dischromic  patches	[4] abnormal menstruation	[5] altered sensorium	[6] irritability	[7] depression	[8] internal itching	[9] passage of gases	[-1] None
1


In [ ]:
print(json.dumps(inputArr))
predictArr=list()
for x in dataset1.columns:
  if x in inputArr.keys():
    predictArr.append(inputArr[x])
  else:
    predictArr.append(0)

{"fatigue": 1, "vomiting": 1, "high_fever": 1, "headache": 1, "back_pain": 1, "skin_rash": 1, "itching": 1}


In [ ]:
if(sum(predictArr)>2):
  tempPred=model.predict([predictArr])
  print('You might have \''+str(tempPred[0])+'\'. You should consult a Doctor')
else:
  print('Not able to Diagnose your Problem')

You might have 'Heart attack'. You should consult a Doctor


In [ ]:
pd.DataFrame(model.predict_proba([predictArr]), columns=model.classes_).max(axis=0)

(vertigo) Paroymsal  Positional Vertigo    0.024392
AIDS                                       0.024391
Acne                                       0.024392
Alcoholic hepatitis                        0.024390
Allergy                                    0.024392
Arthritis                                  0.024390
Bronchial Asthma                           0.024391
Cervical spondylosis                       0.024398
Chicken pox                                0.024389
Chronic cholestasis                        0.024393
Common Cold                                0.024385
Dengue                                     0.024386
Diabetes                                   0.024386
Dimorphic hemmorhoids(piles)               0.024389
Drug Reaction                              0.024394
Fungal infection                           0.024400
GERD                                       0.024391
Gastroenteritis                            0.024393
Heart attack                               0.024393
Hepatitis B 

In [ ]:
pd.DataFrame(model.predict_proba([predictArr]), columns=model.classes_).idxmax(axis=1)

0    Fungal infection
dtype: object